In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [83]:
def setup_download_from_s3( rel_fp, recursive=True ):
    s3_fp = 's3://mousebrainatlas-data/'+rel_fp
    local_fp = os.environ['ROOT_DIR']+rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return 
    
    if recursive:
        ! aws s3 cp --recursive $s3_fp $local_fp
    else:
        ! aws s3 cp $s3_fp $local_fp

In [91]:
setup_download_from_s3('CSHL_patch_samples_features/MD594')

download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/4N/MD594_4N_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/4N/MD594_4N_negative.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/10N/MD594_10N_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/10N/MD594_10N_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/3N/MD594_3N_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/3N/MD594_3N_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/6N/MD594_6N_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/6N/MD594_6N_negative.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/6N/MD594_6N_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/6N/MD594_6N_posi

download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/Sp5C/MD594_Sp5C_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/Sp5C/MD594_Sp5C_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/VCA/MD594_VCA_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/VCA/MD594_VCA_negative.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/Sp5O/MD594_Sp5O_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/Sp5O/MD594_Sp5O_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/VCA/MD594_VCA_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD594/VCA/MD594_VCA_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features/MD594/Tz/MD594_Tz_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD

In [2]:
def setup_upload_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(local_fp, s3_fp))
    else:
        run('aws s3 cp {0} {1}'.format(local_fp, s3_fp))

In [42]:
setup_upload_from_s3('CSHL_data_processed/MD589/Thresholds.pkl', recursive=False)

run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD589/Thresholds.pkl s3://mousebrainatlas-data/CSHL_data_processed/MD589/Thresholds.pkl


In [43]:
import pickle
fn = 'CSHL_data_processed/MD589/Thresholds.pkl'
thresholds = pickle.load(open(os.environ['ROOT_DIR']+fn,'rb'))

In [45]:
thresholds['7nn'][160:170]

[138.4343229125887,
 311.3051776283209,
 535.4004174966207,
 839.4661894072433,
 1297.5713033760658,
 2021.980110052221,
 3305.883348015684,
 5966.536384640669,
 13877.792033371801,
 466532282404860.0]

In [57]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [58]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [4]:
stack = 'MD589'
#yaml_file = 'shape_params-aws.yaml'
yaml_file = 'shape_params.yaml'
img_file = '/CSHL_hsv/'+stack+'/'
scripts_dir = os.environ['REPO_DIR']

In [37]:
Structure.drop()

`kui_diffusionmap`.`structure` (0 tuples)
`kui_diffusionmap`.`__random_patches` (0 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [38]:
@schema
class Structure(dj.Manual):
    definition = """
    structure       : char(10)    # name of mouse brain's structure
    """

In [29]:
@schema
class Structure(dj.Manual):
    definition = """
    id  : int   # sequence
    --------
    structure       : char(10)    # name of mouse brain's structure
    """

In [39]:
paired_structures = ['5N', '6N', '7nn', '7N', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC'] #
all_structures = paired_structures + singular_structures
n=0
for struc in all_structures:
    print("\nAdding "+struc+' to the database')
    Structure.insert1(dict(
                           structure=struc,)
                      ,skip_duplicates=False)
    n+=1


Adding 5N to the database

Adding 6N to the database

Adding 7nn to the database

Adding 7N to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [10]:
structureTable = Structure.fetch(as_dict=True)
strucDF = pd.DataFrame(structureTable)
strucDF

,structure
0,10N
1,12N
2,3N
3,4N
4,5N
5,6N
6,7N
7,7nn
8,Amb
9,AP


In [15]:
Process.drop()

`kui_diffusionmap`.`__process` (4 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [32]:
@schema
class Process(dj.Computed):
    definition="""
    -> Structure
    -----
    image_number : int   #number of images
    """

    bucket = "mousebrainatlas-data"
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        key_item = 'image_number'
        s3_fp = img_file+struc+'/'
        upload_fp = 's3://'+self.bucket+s3_fp
        pkl_out_file = os.environ['ROOT_DIR']+s3_fp
        try:
            objects = os.listdir(pkl_out_file)
            key[key_item] = len(objects)
        except:
            ! python3 $scripts_dir/HSV.py $stack $struc $yaml_file
            ! aws s3 cp --recursive $pkl_out_file $upload_fp
            objects = os.listdir(pkl_out_file)
            key[key_item] = len(objects)
        self.insert1(key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [63]:
Process.populate(reserve_jobs=True) 

populating for  IC


In [64]:
Process.fetch(as_dict=True)

[OrderedDict([('id', 0), ('image_number', 44)]),
 OrderedDict([('id', 1), ('image_number', 5)]),
 OrderedDict([('id', 2), ('image_number', 125)]),
 OrderedDict([('id', 3), ('image_number', 89)]),
 OrderedDict([('id', 4), ('image_number', 13)]),
 OrderedDict([('id', 5), ('image_number', 24)]),
 OrderedDict([('id', 6), ('image_number', 76)]),
 OrderedDict([('id', 7), ('image_number', 100)]),
 OrderedDict([('id', 8), ('image_number', 50)]),
 OrderedDict([('id', 9), ('image_number', 69)]),
 OrderedDict([('id', 10), ('image_number', 57)]),
 OrderedDict([('id', 11), ('image_number', 80)]),
 OrderedDict([('id', 12), ('image_number', 126)]),
 OrderedDict([('id', 13), ('image_number', 24)]),
 OrderedDict([('id', 14), ('image_number', 11)]),
 OrderedDict([('id', 15), ('image_number', 73)]),
 OrderedDict([('id', 16), ('image_number', 67)]),
 OrderedDict([('id', 17), ('image_number', 141)]),
 OrderedDict([('id', 18), ('image_number', 24)]),
 OrderedDict([('id', 19), ('image_number', 45)]),
 Ordere

In [61]:
(schema.jobs & 'status="reserved"' ).delete()

In [85]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__sample_features,13a831874352b548ac7b5e63a860aa1b,reserved,=BLOB=,,=BLOB=,kui@172.31.26.225,ip-172-31-26-225,27971,23342,2019-07-25 12:30:32
__sample_features,57aab2fcd93938cfbca356bb5571b25b,reserved,=BLOB=,,=BLOB=,kui@172.31.31.55,ip-172-31-31-55,624,23335,2019-07-25 12:26:13
__sample_features,647b2a336fa813f6b4bee8e561b090c4,reserved,=BLOB=,,=BLOB=,kui@172.31.28.40,ip-172-31-28-40,6489,23332,2019-07-25 12:33:44
__sample_features,66e96423819dd99274f4a2c2ebc02f61,reserved,=BLOB=,,=BLOB=,kui@172.31.28.191,ip-172-31-28-191,27166,23334,2019-07-25 12:33:34
__sample_features,6a65edb0cc17d66c677814115b1477f5,reserved,=BLOB=,,=BLOB=,kui@172.31.17.36,ip-172-31-17-36,28009,23340,2019-07-25 12:25:34
__sample_features,6a7698388ce6434a04602d72ed1eb955,reserved,=BLOB=,,=BLOB=,kui@172.31.18.31,ip-172-31-18-31,4130,23331,2019-07-25 12:29:24
__sample_features,71a8a02dc1c7e9911c21fdc5bba63db0,reserved,=BLOB=,,=BLOB=,kui@172.31.30.71,ip-172-31-30-71,3243,23338,2019-07-25 12:30:08
__sample_features,8dc5367d65459f69d78eb03e93f7703f,reserved,=BLOB=,,=BLOB=,kui@172.31.20.146,ip-172-31-20-146,15138,23341,2019-07-25 12:26:24
__sample_features,8eee1505bf48577ff9c0274e241f0da6,reserved,=BLOB=,,=BLOB=,kui@172.31.27.82,ip-172-31-27-82,3057,23337,2019-07-25 12:30:11
__sample_features,d3d85f072985af3dd2cc517706390cc1,reserved,=BLOB=,,=BLOB=,kui@172.31.26.212,ip-172-31-26-212,32261,23333,2019-07-25 12:31:31


In [40]:
Structure.fetch(as_dict=True)

[OrderedDict([('structure', '10N')]),
 OrderedDict([('structure', '12N')]),
 OrderedDict([('structure', '3N')]),
 OrderedDict([('structure', '4N')]),
 OrderedDict([('structure', '5N')]),
 OrderedDict([('structure', '6N')]),
 OrderedDict([('structure', '7N')]),
 OrderedDict([('structure', '7nn')]),
 OrderedDict([('structure', 'Amb')]),
 OrderedDict([('structure', 'AP')]),
 OrderedDict([('structure', 'DC')]),
 OrderedDict([('structure', 'IC')]),
 OrderedDict([('structure', 'LC')]),
 OrderedDict([('structure', 'LRt')]),
 OrderedDict([('structure', 'PBG')]),
 OrderedDict([('structure', 'Pn')]),
 OrderedDict([('structure', 'RMC')]),
 OrderedDict([('structure', 'RtTg')]),
 OrderedDict([('structure', 'SC')]),
 OrderedDict([('structure', 'SNC')]),
 OrderedDict([('structure', 'SNR')]),
 OrderedDict([('structure', 'Sp5C')]),
 OrderedDict([('structure', 'Sp5I')]),
 OrderedDict([('structure', 'Sp5O')]),
 OrderedDict([('structure', 'Tz')]),
 OrderedDict([('structure', 'VCA')]),
 OrderedDict([('stru

In [35]:
RandomPatches.drop()

`kui_diffusionmap`.`__random_patches` (28 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [77]:
SampleFeatures.drop()

`kui_diffusionmap`.`__sample_features` (28 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [52]:
! python Sample_features_datajoint.py 'local' 'MD589'

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
Connecting kui@ucsd-demo-db.datajoint.io:3306
./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
populating for  10N
CSHL_patch_samples_features/MD589/10N/MD589_10N_positive.pkl
CSHL_patch_samples_features/MD589/10N/MD589_10N_negative.pkl
run cmd= python3 /Users/kuiqian/Github/shapeology_code/scripts//Cell_generator.py MD589 10N negative shape_params.yaml CSHL_patch_samples
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is depreca

(pid=5973) Completed 674.4 KiB/~13.7 MiB (1.2 MiB/s) with ~378 file(s) remaining (calculating...)
(pid=5973) Completed 708.1 KiB/~13.7 MiB (1.3 MiB/s) with ~378 file(s) remaining (calculating...)
(pid=5973) Completed 743.3 KiB/~13.8 MiB (1.3 MiB/s) with ~382 file(s) remaining (calculating...)
(pid=5973) Completed 773.3 KiB/~13.8 MiB (1.4 MiB/s) with ~383 file(s) remaining (calculating...)
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/203_3.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/10N_surround_500um_noclass/203_3.tif
(pid=5973) Completed 773.3 KiB/~13.8 MiB (1.4 MiB/s) with ~382 file(s) remaining (calculating...)
(pid=5973) Completed 809.4 KiB/~13.9 MiB (1.4 MiB/s) with ~385 file(s) remaining (calculating...)
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/203_4.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/10N_surround_

(pid=5973) Completed 1.8 MiB/15.9 MiB (2.2 MiB/s) with 409 file(s) remaining
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/206_2.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/10N_surround_500um_noclass/206_2.tif
(pid=5973) Completed 1.8 MiB/15.9 MiB (2.2 MiB/s) with 408 file(s) remaining
(pid=5973) Completed 1.8 MiB/15.9 MiB (2.2 MiB/s) with 408 file(s) remaining
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/206_3.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/10N_surround_500um_noclass/206_3.tif
(pid=5973) Completed 1.8 MiB/15.9 MiB (2.2 MiB/s) with 407 file(s) remaining
(pid=5973) Completed 1.9 MiB/15.9 MiB (2.2 MiB/s) with 407 file(s) remaining
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/206_1.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD58

(pid=5973) Completed 2.5 MiB/15.9 MiB (2.6 MiB/s) with 389 file(s) remaining
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/206_9.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/10N_surround_500um_noclass/206_9.tif
(pid=5973) Completed 2.5 MiB/15.9 MiB (2.6 MiB/s) with 388 file(s) remaining
(pid=5973) Completed 2.5 MiB/15.9 MiB (2.6 MiB/s) with 388 file(s) remaining
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/208_5.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/10N_surround_500um_noclass/208_5.tif
(pid=5973) Completed 2.5 MiB/15.9 MiB (2.6 MiB/s) with 387 file(s) remaining
(pid=5973) Completed 2.6 MiB/15.9 MiB (2.6 MiB/s) with 387 file(s) remaining
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/208_4.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD58

(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/211_6.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/10N_surround_500um_noclass/211_6.tif
(pid=5973) Completed 3.6 MiB/15.9 MiB (3.0 MiB/s) with 358 file(s) remaining
(pid=5973) Completed 3.6 MiB/15.9 MiB (3.1 MiB/s) with 358 file(s) remaining
(pid=5973) Completed 3.6 MiB/15.9 MiB (3.1 MiB/s) with 358 file(s) remaining
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/211_0.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/10N_surround_500um_noclass/211_0.tif
(pid=5973) Completed 3.6 MiB/15.9 MiB (3.1 MiB/s) with 357 file(s) remaining
(pid=5973) Completed 3.7 MiB/15.9 MiB (3.1 MiB/s) with 357 file(s) remaining
(pid=5973) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/10N_surround_500um_noclass/211_5.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD58

...

(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/234_3.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/234_3.tif
(pid=6082) Completed 9.6 MiB/13.6 MiB (4.3 MiB/s) with 114 file(s) remaining
(pid=6082) Completed 9.6 MiB/13.6 MiB (4.3 MiB/s) with 114 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/234_5.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/234_5.tif
(pid=6082) Completed 9.6 MiB/13.6 MiB (4.3 MiB/s) with 113 file(s) remaining
(pid=6082) Completed 9.7 MiB/13.6 MiB (4.3 MiB/s) with 113 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/234_6.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/234_6.tif
(pid=6082) Completed 9.7 MiB/13.6 MiB (4.3 MiB/s) with 112 file(s) remaining
(pid=6082) Completed 9.7 MiB/13.6 MiB (4.3 MiB/s) with 112 file(s) remaining
(pid=6082) download: s

(pid=6082) Completed 10.5 MiB/13.6 MiB (4.3 MiB/s) with 91 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/237_1.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/237_1.tif
(pid=6082) Completed 10.5 MiB/13.6 MiB (4.3 MiB/s) with 90 file(s) remaining
(pid=6082) Completed 10.5 MiB/13.6 MiB (4.3 MiB/s) with 90 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/237_0.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/237_0.tif
(pid=6082) Completed 10.5 MiB/13.6 MiB (4.3 MiB/s) with 89 file(s) remaining
(pid=6082) Completed 10.5 MiB/13.6 MiB (4.3 MiB/s) with 89 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/237_4.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/237_4.tif
(pid=6082) Completed 10.5 MiB/13.6 MiB (4.3 MiB/s) with 88 file(s) remaining
(pid=6082) Completed 1

(pid=6082) Completed 11.5 MiB/13.6 MiB (4.4 MiB/s) with 62 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/239_9.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/239_9.tif
(pid=6082) Completed 11.5 MiB/13.6 MiB (4.4 MiB/s) with 61 file(s) remaining
(pid=6082) Completed 11.5 MiB/13.6 MiB (4.4 MiB/s) with 61 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/240_0.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/240_0.tif
(pid=6082) Completed 11.5 MiB/13.6 MiB (4.4 MiB/s) with 60 file(s) remaining
(pid=6082) Completed 11.5 MiB/13.6 MiB (4.4 MiB/s) with 60 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/240_5.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/240_5.tif
(pid=6082) Completed 11.5 MiB/13.6 MiB (4.4 MiB/s) with 59 file(s) remaining
(pid=6082) Completed 1

(pid=6082) Completed 12.3 MiB/13.6 MiB (4.4 MiB/s) with 39 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/242_3.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/242_3.tif
(pid=6082) Completed 12.3 MiB/13.6 MiB (4.4 MiB/s) with 38 file(s) remaining
(pid=6082) Completed 12.3 MiB/13.6 MiB (4.4 MiB/s) with 38 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/242_2.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/242_2.tif
(pid=6082) Completed 12.3 MiB/13.6 MiB (4.4 MiB/s) with 37 file(s) remaining
(pid=6082) Completed 12.3 MiB/13.6 MiB (4.4 MiB/s) with 37 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/242_1.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/242_1.tif
(pid=6082) Completed 12.3 MiB/13.6 MiB (4.4 MiB/s) with 36 file(s) remaining
(pid=6082) Completed 1

(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/244_8.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/244_8.tif
(pid=6082) Completed 13.3 MiB/13.6 MiB (4.4 MiB/s) with 10 file(s) remaining
(pid=6082) Completed 13.3 MiB/13.6 MiB (4.4 MiB/s) with 10 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/244_9.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/244_9.tif
(pid=6082) Completed 13.3 MiB/13.6 MiB (4.4 MiB/s) with 9 file(s) remaining
(pid=6082) Completed 13.3 MiB/13.6 MiB (4.4 MiB/s) with 9 file(s) remaining
(pid=6082) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/12N/245_2.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/12N/245_2.tif
(pid=6082) Completed 13.3 MiB/13.6 MiB (4.4 MiB/s) with 8 file(s) remaining
(pid=6082) Completed 13.4 MiB/13.6 MiB (4.4 MiB/s) with 8 file(s) remaining
(pid=6082) download: s3://

In [46]:
RandomPatches.fetch(as_dict=True)

[OrderedDict([('structure', '10N'),
              ('positive_patches_number', 271),
              ('negative_patches_number', 330)]),
 OrderedDict([('structure', '12N'),
              ('positive_patches_number', 390),
              ('negative_patches_number', 390)]),
 OrderedDict([('structure', '3N'),
              ('positive_patches_number', 230),
              ('negative_patches_number', 230)]),
 OrderedDict([('structure', '4N'),
              ('positive_patches_number', 113),
              ('negative_patches_number', 140)]),
 OrderedDict([('structure', '5N'),
              ('positive_patches_number', 479),
              ('negative_patches_number', 480)]),
 OrderedDict([('structure', '6N'),
              ('positive_patches_number', 216),
              ('negative_patches_number', 270)]),
 OrderedDict([('structure', '7N'),
              ('positive_patches_number', 826),
              ('negative_patches_number', 830)]),
 OrderedDict([('structure', '7nn'),
              ('positive_patche

In [90]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [65]:
(schema.jobs & 'status="error"').delete()

In [88]:
SampleFeatures.fetch(as_dict=True)

[OrderedDict([('structure', '10N'),
              ('size_of_positive', 1682),
              ('size_of_negative', 1926)]),
 OrderedDict([('structure', '12N'),
              ('size_of_positive', 2421),
              ('size_of_negative', 2345)]),
 OrderedDict([('structure', '3N'),
              ('size_of_positive', 1427),
              ('size_of_negative', 1421)]),
 OrderedDict([('structure', '4N'),
              ('size_of_positive', 701),
              ('size_of_negative', 868)]),
 OrderedDict([('structure', '5N'),
              ('size_of_positive', 2973),
              ('size_of_negative', 2980)]),
 OrderedDict([('structure', '6N'),
              ('size_of_positive', 1340),
              ('size_of_negative', 1629)]),
 OrderedDict([('structure', '7N'),
              ('size_of_positive', 5128),
              ('size_of_negative', 5130)]),
 OrderedDict([('structure', '7nn'),
              ('size_of_positive', 5124),
              ('size_of_negative', 6162)]),
 OrderedDict([('structure', 'Am